In [7]:
import pandas as pd 

In [8]:
df = pd.read_csv('../../datasets/OnlineRetail.csv',encoding='latin-1')

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [9]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [10]:
df.shape

(541909, 8)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [12]:
df.dropna(subset= ['Description'],inplace=True)
df['Description'] = df['Description'].apply(lambda d: d.lower().strip())

In [13]:
df.sort_values('InvoiceNo',inplace=True)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,white hanging heart t-light holder,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,white metal lantern,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,cream cupid hearts coat hanger,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,knitted union flag hot water bottle,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,red woolly hottie white heart.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,zinc t-light holder stars small,-11,12/9/2011 9:57,0.83,14397.0,United Kingdom
541541,C581499,M,manual,-1,12/9/2011 10:28,224.69,15498.0,United Kingdom
541715,C581568,21258,victorian sewing box large,-5,12/9/2011 11:57,10.95,15311.0,United Kingdom
541716,C581569,84978,hanging heart jar t-light holder,-1,12/9/2011 11:58,1.25,17315.0,United Kingdom


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 540455 entries, 0 to 541717
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    540455 non-null  object 
 1   StockCode    540455 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     540455 non-null  int64  
 4   InvoiceDate  540455 non-null  object 
 5   UnitPrice    540455 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      540455 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 37.1+ MB


In [15]:
df.sort_values('InvoiceNo',inplace=True)

In [16]:
def clean_invoice(item):
    if not item.isnumeric():
        return float(item[1:])
    return float(item)
df['InvoiceNo']=df['InvoiceNo'].apply(clean_invoice)

In [28]:
francedf = df[df.Country == 'UK'].groupby(['InvoiceNo','Description'])['Quantity'].sum()

In [29]:
francedf

Series([], Name: Quantity, dtype: int64)

In [30]:
france_basket = francedf.unstack()

In [31]:
france_basket = france_basket.fillna(0)

In [32]:
france_basket

Description
InvoiceNo


In [33]:
def hot_encode(val):
    if val <= 0:
        return 0
    return 1

france_basket = france_basket.applymap(hot_encode)

### build model with apriori

In [34]:
from mlxtend.frequent_patterns import apriori, association_rules

In [37]:
item_frequent = apriori(france_basket,min_support=0.05,use_colnames=True,)
item_frequent.head()

,support,itemsets


In [38]:
rules = association_rules(item_frequent,metric='lift',min_threshold=1)
rules.head()

ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set

In [26]:
rules.sort_values(['lift','confidence'],ascending=[False,False],inplace=True)

In [27]:
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
15,(childrens cutlery spaceboy),(childrens cutlery dolly girl),0.058568,0.060738,0.054230,0.925926,15.244709,0.050673,12.680043
14,(childrens cutlery dolly girl),(childrens cutlery spaceboy),0.060738,0.058568,0.054230,0.892857,15.244709,0.050673,8.786696
114,"(alarm clock bakelike green, alarm clock bakel...",(alarm clock bakelike red),0.062907,0.080260,0.054230,0.862069,10.740913,0.049181,6.668113
115,(alarm clock bakelike red),"(alarm clock bakelike green, alarm clock bakel...",0.080260,0.062907,0.054230,0.675676,10.740913,0.049181,2.889371
126,"(postage, alarm clock bakelike green)",(alarm clock bakelike red),0.071584,0.080260,0.060738,0.848485,10.571663,0.054992,6.070282
127,(alarm clock bakelike red),"(postage, alarm clock bakelike green)",0.080260,0.071584,0.060738,0.756757,10.571663,0.054992,3.816823
113,"(alarm clock bakelike red, alarm clock bakelik...",(alarm clock bakelike green),0.062907,0.082430,0.054230,0.862069,10.458258,0.049045,6.652386
116,(alarm clock bakelike green),"(alarm clock bakelike red, alarm clock bakelik...",0.082430,0.062907,0.054230,0.657895,10.458258,0.049045,2.739196
2,(alarm clock bakelike red),(alarm clock bakelike green),0.080260,0.082430,0.067245,0.837838,10.164296,0.060629,5.658351
3,(alarm clock bakelike green),(alarm clock bakelike red),0.082430,0.080260,0.067245,0.815789,10.164296,0.060629,4.992873


4
5
6
7
